In [1]:
import os
os.environ['SPARK_NAME'] = "/opt/spark"
# os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
# os.environ['PYSPARK_PYTHON'] = 'python'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/opt/spark/work-dir/venv/bin/python3'
os.environ['PYSPARK_PYTHON'] = '/opt/spark/work-dir/venv/bin/python3'

# można też spróbować wykorzystać moduł findspark do automatycznego odnalezienia miejsca instalacji sparka
# import findspark
# findspark.init()
# lub
# findspark.init("/opt/spark")

from pyspark.sql import SparkSession

# spark = SparkSession.builder.master("spark://spark-master:7077").appName("Create-DataFrame").getOrCreate()
# konfiguracja z określeniem liczby wątków (2) oraz ilości pamięci do wykorzystania poza stertą interpretera Pythona
spark = SparkSession\
        .builder\
        .master("local[24]")\
        .appName("Create-DataFrame")\
        .config("spark.memory.offHeap.enabled","true")\
        .config("spark.memory.offHeap.size","48g")\
        .getOrCreate()
spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/28 23:27:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<SparkContext master=local[24] appName=Create-DataFrame>

In [2]:
# dostosuj ścieżkę do pliku do swoich danych, tutaj został utworzony mniejszy zbiór niż w poprzednim labie
df = spark.read.csv('./employee.csv', header=True, inferSchema=True)

In [3]:
# tworzymy widok tymczasowy w pamięci węzła
df.createOrReplaceTempView("EMPLOYEE_DATA")

In [4]:
# pobranie danych jak z tabeli SQL
spark.sql("Select * from EMPLOYEE_DATA limit 4").show()
spark.sql("select firstname from EMPLOYEE_DATA").show(10)


+---+---------+-------------------+---+-------+
| id|firstname|           lastname|age| salary|
+---+---------+-------------------+---+-------+
|  1|Krzysztof|       Mieczykowski| 31|7997.69|
|  2| Zbigniew|         Malinowski| 26|7493.97|
|  3|Krzysztof|             Wlotka| 19|9745.33|
|  4|    Agata|Brzęczyszczykiewicz| 43|7968.02|
+---+---------+-------------------+---+-------+

+----------+
| firstname|
+----------+
| Krzysztof|
|  Zbigniew|
| Krzysztof|
|     Agata|
| Krzysztof|
|   Wisława|
|      Adam|
|  Zbigniew|
|Mieczysław|
|     Agata|
+----------+
only showing top 10 rows



In [5]:
spark.sql("select firstname, count(firstname), avg(salary) from EMPLOYEE_DATA group by firstname").show()

[Stage 4:========================================>                (17 + 7) / 24]

+----------+----------------+------------------+
| firstname|count(firstname)|       avg(salary)|
+----------+----------------+------------------+
|   Wisława|         1997928| 7849.999004818989|
|Mieczysław|         2000802| 7850.031071265435|
|     Agata|         2000505|7850.2951651757885|
| Krzysztof|         1999775| 7849.843396027059|
|     Marek|         2001653| 7849.047706855308|
|      Adam|         2001070| 7850.422978231653|
| Katarzyna|         1998606| 7849.733991917356|
|  Wojciech|         1999107| 7849.887235455639|
|  Zbigniew|         2001392| 7849.732310666769|
|Aleksandra|         1999162| 7849.634000641244|
+----------+----------------+------------------+



In [6]:
rising = 0.1 # 10% podwyżki
spark.sql(f"select firstname, lastname, salary, round(salary + salary * {rising},2) as after_rising from EMPLOYEE_DATA").show(5)


+---------+-------------------+-------+------------+
|firstname|           lastname| salary|after_rising|
+---------+-------------------+-------+------------+
|Krzysztof|       Mieczykowski|7997.69|     8797.46|
| Zbigniew|         Malinowski|7493.97|     8243.37|
|Krzysztof|             Wlotka|9745.33|    10719.86|
|    Agata|Brzęczyszczykiewicz|7968.02|     8764.82|
|Krzysztof|             Szczaw|8192.67|     9011.94|
+---------+-------------------+-------+------------+
only showing top 5 rows



In [7]:
spark.catalog.currentCatalog()

'spark_catalog'

In [8]:
# dla zrealizowania kolejnych przykładów dokonamy kilku modyfikacji pliku employee
# 1. dodanie kolumny ID - indeksu
from pyspark.sql.functions import monotonically_increasing_id

df = df.withColumn("ID", monotonically_increasing_id())


In [9]:
df.show(10)

+---+----------+-------------------+---+-------+
| ID| firstname|           lastname|age| salary|
+---+----------+-------------------+---+-------+
|  0| Krzysztof|       Mieczykowski| 31|7997.69|
|  1|  Zbigniew|         Malinowski| 26|7493.97|
|  2| Krzysztof|             Wlotka| 19|9745.33|
|  3|     Agata|Brzęczyszczykiewicz| 43|7968.02|
|  4| Krzysztof|             Szczaw| 39|8192.67|
|  5|   Wisława|               Glut| 35|8441.91|
|  6|      Adam|Brzęczyszczykiewicz| 60|7023.85|
|  7|  Zbigniew|              Pysla| 67|6176.82|
|  8|Mieczysław|Brzęczyszczykiewicz| 48|7611.33|
|  9|     Agata|       Mieczykowski| 26|8180.23|
+---+----------+-------------------+---+-------+
only showing top 10 rows



In [10]:
# dokonamy podziału danych i zapisania w różnych formatach
splits = df.randomSplit(weights=[0.3, 0.7], seed=19)


In [11]:
splits[0].count(), splits[1].count()

(5999901, 14000099)

In [12]:
splits[1].createOrReplaceTempView("EMPLOYEE_DATA_SPLIT_1")

In [13]:
splits[0].write.json('employee_data.json', mode='overwrite')

In [14]:
!ls ./employee_data.json/*.json

./employee_data.json/part-00000-d9f2f5cd-7e83-4e5d-aed8-e07f81ef1cae-c000.json
./employee_data.json/part-00001-d9f2f5cd-7e83-4e5d-aed8-e07f81ef1cae-c000.json
./employee_data.json/part-00002-d9f2f5cd-7e83-4e5d-aed8-e07f81ef1cae-c000.json
./employee_data.json/part-00003-d9f2f5cd-7e83-4e5d-aed8-e07f81ef1cae-c000.json
./employee_data.json/part-00004-d9f2f5cd-7e83-4e5d-aed8-e07f81ef1cae-c000.json
./employee_data.json/part-00005-d9f2f5cd-7e83-4e5d-aed8-e07f81ef1cae-c000.json
./employee_data.json/part-00006-d9f2f5cd-7e83-4e5d-aed8-e07f81ef1cae-c000.json
./employee_data.json/part-00007-d9f2f5cd-7e83-4e5d-aed8-e07f81ef1cae-c000.json
./employee_data.json/part-00008-d9f2f5cd-7e83-4e5d-aed8-e07f81ef1cae-c000.json
./employee_data.json/part-00009-d9f2f5cd-7e83-4e5d-aed8-e07f81ef1cae-c000.json
./employee_data.json/part-00010-d9f2f5cd-7e83-4e5d-aed8-e07f81ef1cae-c000.json
./employee_data.json/part-00011-d9f2f5cd-7e83-4e5d-aed8-e07f81ef1cae-c000.json
./employee_data.json/part-00012-d9f2f5cd-7e83-4e5d-a

In [15]:
# aby móc wykorzystać dane w przykładach ze złączaniem, zapiszemy jeszcze próbkę danych z głównej ramki
# z identyfikatorami oraz dodatkową kolumną z podwyżką
from pyspark.sql.functions import col, lit, round

lucky_guys = spark.sql("select * from EMPLOYEE_DATA").sample(0.01)\
.withColumn('rising', lit('10%')).withColumn('salary after rising', round(col('salary') * 1.1, 2))

In [22]:
lucky_guys.write.mode('overwrite').saveAsTable("lucky_employees2", format='parquet')

25/01/29 00:13:38 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/01/29 00:13:38 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/01/29 00:13:38 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/01/29 00:13:38 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/01/29 00:13:38 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
25/01/29 00:13:38 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 58.46% for 13 writers
25/01/29 00:13:38 WARN MemoryManager: Total allocation exceeds 95.

In [23]:
spark.catalog.listTables()

[Table(name='lucky_employees2', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='EMPLOYEE_DATA', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='EMPLOYEE_DATA_SPLIT_1', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [26]:
!ls ./spark-warehouse/lucky_employees/*.parquet

./spark-warehouse/lucky_employees/part-00000-3ff5e378-9021-4466-a7d8-b67183e0520e-c000.snappy.parquet
./spark-warehouse/lucky_employees/part-00001-3ff5e378-9021-4466-a7d8-b67183e0520e-c000.snappy.parquet
./spark-warehouse/lucky_employees/part-00002-3ff5e378-9021-4466-a7d8-b67183e0520e-c000.snappy.parquet
./spark-warehouse/lucky_employees/part-00003-3ff5e378-9021-4466-a7d8-b67183e0520e-c000.snappy.parquet
./spark-warehouse/lucky_employees/part-00004-3ff5e378-9021-4466-a7d8-b67183e0520e-c000.snappy.parquet
./spark-warehouse/lucky_employees/part-00005-3ff5e378-9021-4466-a7d8-b67183e0520e-c000.snappy.parquet
./spark-warehouse/lucky_employees/part-00006-3ff5e378-9021-4466-a7d8-b67183e0520e-c000.snappy.parquet
./spark-warehouse/lucky_employees/part-00007-3ff5e378-9021-4466-a7d8-b67183e0520e-c000.snappy.parquet
./spark-warehouse/lucky_employees/part-00008-3ff5e378-9021-4466-a7d8-b67183e0520e-c000.snappy.parquet
./spark-warehouse/lucky_employees/part-00009-3ff5e378-9021-4466-a7d8-b67183e0520e-

In [28]:
# przykład złączania danych na różnych źródłach danych
# zapytanie SQL bezpośrednio na plikach - tutaj zapisanych wcześniej JSON-ach oraz parquet
query = """
SELECT ed.ID, ed.firstname, ed.lastname, ed.salary, lucky.rising, lucky.`salary after rising`
FROM json.`./employee_data.json/` as jtable 
join EMPLOYEE_DATA ed on jtable.ID=ed.ID 
join parquet.`./spark-warehouse/lucky_employees/` as lucky on ed.ID=lucky.ID
"""
df_from_json = spark.sql(query).show(10)

[Stage 38:===================================================>    (22 + 2) / 24]

+-----+----------+-------------------+-------+------+-------------------+
|   ID| firstname|           lastname| salary|rising|salary after rising|
+-----+----------+-------------------+-------+------+-------------------+
| 6700|      Adam|              Pysla|7899.21|   10%|            8689.13|
|12390| Krzysztof|       Mieczykowski|6077.47|   10%|            6685.22|
|23766|   Wisława|               Glut|6990.86|   10%|            7689.95|
|33878| Krzysztof|              Pysla|7148.47|   10%|            7863.32|
|38510|Aleksandra|Brzęczyszczykiewicz|7690.41|   10%|            8459.45|
|56130|  Zbigniew|               Glut|7025.08|   10%|            7727.59|
|69677|     Agata|               Glut|7119.91|   10%|             7831.9|
|74000|      Adam|             Szczaw|8488.38|   10%|            9337.22|
|75630|Mieczysław|         Malinowski|7563.36|   10%|             8319.7|
|83281|     Agata|              Pysla|9092.46|   10%|           10001.71|
+-----+----------+-------------------+

In [29]:
df.write.bucketBy(16, 'ID').mode('overwrite').sortBy('salary').saveAsTable('employee_id_bucketed')

25/01/29 00:14:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/01/29 00:14:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/01/29 00:14:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/01/29 00:14:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/01/29 00:14:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/01/29 00:14:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/01/29 00:14:57 WARN MemoryManager: Total allocation exceeds 95.00

In [32]:
!ls spark-warehouse/employee_id_bucketed/*.parquet

spark-warehouse/employee_id_bucketed/part-00000-0517a2dd-fe69-467a-a3a1-ed210ca9adf2_00000.c000.snappy.parquet
spark-warehouse/employee_id_bucketed/part-00000-0517a2dd-fe69-467a-a3a1-ed210ca9adf2_00001.c000.snappy.parquet
spark-warehouse/employee_id_bucketed/part-00000-0517a2dd-fe69-467a-a3a1-ed210ca9adf2_00002.c000.snappy.parquet
spark-warehouse/employee_id_bucketed/part-00000-0517a2dd-fe69-467a-a3a1-ed210ca9adf2_00003.c000.snappy.parquet
spark-warehouse/employee_id_bucketed/part-00000-0517a2dd-fe69-467a-a3a1-ed210ca9adf2_00004.c000.snappy.parquet
spark-warehouse/employee_id_bucketed/part-00000-0517a2dd-fe69-467a-a3a1-ed210ca9adf2_00005.c000.snappy.parquet
spark-warehouse/employee_id_bucketed/part-00000-0517a2dd-fe69-467a-a3a1-ed210ca9adf2_00006.c000.snappy.parquet
spark-warehouse/employee_id_bucketed/part-00000-0517a2dd-fe69-467a-a3a1-ed210ca9adf2_00007.c000.snappy.parquet
spark-warehouse/employee_id_bucketed/part-00000-0517a2dd-fe69-467a-a3a1-ed210ca9adf2_00008.c000.snappy.parquet
s

In [33]:
spark.table('employee_id_bucketed').show(10)

+------+----------+-------------------+---+-------+
|    ID| firstname|           lastname|age| salary|
+------+----------+-------------------+---+-------+
|717307|   Wisława|         Wróblewski| 39|3741.55|
| 95660|Mieczysław|           Barański| 18|3800.22|
|692074| Katarzyna|         Wróblewski| 51|3993.93|
|325549| Katarzyna|Brzęczyszczykiewicz| 20|4034.38|
|613370| Krzysztof|           Barański| 50|4092.33|
| 24585|Mieczysław|              Pysla| 56|4206.94|
|334782|  Zbigniew|         Wróblewski| 62|4228.71|
|488433|  Zbigniew|             Wlotka| 63|4233.98|
|529790|  Wojciech|               Glut| 51|4248.47|
|328475|  Zbigniew|             Szczaw| 48|4279.91|
+------+----------+-------------------+---+-------+
only showing top 10 rows



In [34]:
spark.catalog.listTables()

[Table(name='employee_id_bucketed', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='lucky_employees2', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='EMPLOYEE_DATA', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='EMPLOYEE_DATA_SPLIT_1', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [37]:
spark.sql('DROP TABLE employee_id_bucketed')

DataFrame[]

In [38]:
df.write.partitionBy("lastname").mode('overwrite').saveAsTable("employees_partitioned_lastname")

25/01/29 00:17:02 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/01/29 00:17:02 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/01/29 00:17:02 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/01/29 00:17:02 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/01/29 00:17:02 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
25/01/29 00:17:02 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/01/29 00:17:02 WARN MemoryManager: Total allocation exceeds 95.

In [39]:
buckets = spark.sql("select distinct firstname from EMPLOYEE_DATA").count()
buckets

                                                                                



10

In [41]:
!ls spark-warehouse/employees_partitioned_lastname

'lastname=Barański'		'lastname=Malinowski'	 'lastname=Wlotka'
'lastname=Brzęczyszczykiewicz'	'lastname=Mieczykowski'  'lastname=Wróblewski'
'lastname=Glut'			'lastname=Pysla'	  _SUCCESS
'lastname=Kowalski'		'lastname=Szczaw'


In [42]:
df.filter(df.lastname == 'Pysla').groupby('lastname').agg({'salary': 'avg'}).explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[lastname#19], functions=[avg(salary#21)])
   +- Exchange hashpartitioning(lastname#19, 200), ENSURE_REQUIREMENTS, [plan_id=994]
      +- HashAggregate(keys=[lastname#19], functions=[partial_avg(salary#21)])
         +- Filter (isnotnull(lastname#19) AND (lastname#19 = Pysla))
            +- FileScan csv [lastname#19,salary#21] Batched: false, DataFilters: [isnotnull(lastname#19), (lastname#19 = Pysla)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/opt/spark/work-dir/employee.csv], PartitionFilters: [], PushedFilters: [IsNotNull(lastname), EqualTo(lastname,Pysla)], ReadSchema: struct<lastname:string,salary:double>




In [43]:
%%time
df.filter(df.lastname == 'Pysla').groupby('lastname').agg({'salary': 'avg'}).show(10)

[Stage 57:===================================================>    (22 + 2) / 24]

+--------+-----------------+
|lastname|      avg(salary)|
+--------+-----------------+
|   Pysla|7849.279204497448|
+--------+-----------------+

CPU times: user 8.51 ms, sys: 7.35 ms, total: 15.9 ms
Wall time: 1.99 s


In [44]:
spark.sql("select lastname, avg(salary) from employees_partitioned_lastname where lastname='Pysla' group by lastname").explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[lastname#590], functions=[avg(salary#589)])
   +- Exchange hashpartitioning(lastname#590, 200), ENSURE_REQUIREMENTS, [plan_id=1064]
      +- HashAggregate(keys=[lastname#590], functions=[partial_avg(salary#589)])
         +- FileScan parquet spark_catalog.default.employees_partitioned_lastname[salary#589,lastname#590] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/opt/spark/work-dir/spark-warehouse/employees_partitioned_lastnam..., PartitionFilters: [isnotnull(lastname#590), (lastname#590 = Pysla)], PushedFilters: [], ReadSchema: struct<salary:double>




In [46]:
%%time
spark.sql("select lastname, avg(salary) from employees_partitioned_lastname where lastname='Pysla' group by lastname").show()

+--------+-----------------+
|lastname|      avg(salary)|
+--------+-----------------+
|   Pysla|7849.279204497448|
+--------+-----------------+

CPU times: user 6.8 ms, sys: 1.3 ms, total: 8.1 ms
Wall time: 522 ms


In [47]:
spark.sparkContext.stop()

In [59]:


spark = SparkSession.builder.appName("Zamowienia").getOrCreate()

schema = StructType([
    StructField("Kraj", StringType(), True),
    StructField("Sprzedawca", StringType(), True),
    StructField("Data zamowienia", StringType(), True),
    StructField("idZamowienia", StringType(), True),
    StructField("Utarg", StringType(), True)
])

df = spark.read.option("delimiter", ";").csv("zamowienia.txt", header=True, schema=schema)
df = df.withColumn("Sprzedawca", regexp_replace(col("Sprzedawca"), r'[^\x00-\x7F]', ''))
df = df.withColumn("Utarg", regexp_replace(col("Utarg"), r"[^0-9,.]", ""))
df = df.withColumn("Utarg", regexp_replace(col("Utarg"), r",", "."))
df = df.withColumn("Utarg", col("Utarg").cast(DoubleType()))
df = df.withColumnRenamed("idZamowienia", "idZamowienia").withColumn("idZamowienia", col("idZamowienia").cast(IntegerType()))


df.show()
df.printSchema()


+------+----------+---------------+------------+-------+
|  Kraj|Sprzedawca|Data zamowienia|idZamowienia|  Utarg|
+------+----------+---------------+------------+-------+
|Polska|  Kowalski|     16.07.2003|       10248|  440.0|
|Polska|   Sowiski|     10.07.2003|       10249| 1863.4|
|Niemcy|   Peacock|     12.07.2003|       10250| 1552.6|
|Niemcy| Leverling|     15.07.2003|       10251| 654.06|
|Niemcy|   Peacock|     11.07.2003|       10252| 3597.9|
|Niemcy| Leverling|     16.07.2003|       10253| 1444.8|
|Polska|  Kowalski|     23.07.2003|       10254| 556.62|
|Polska|     Dudek|     15.07.2003|       10255| 2490.5|
|Niemcy| Leverling|     17.07.2003|       10256|  517.8|
|Niemcy|   Peacock|     22.07.2003|       10257| 1119.9|
|Niemcy|   Davolio|     23.07.2003|       10258|1614.88|
|Niemcy|   Peacock|     25.07.2003|       10259|  100.8|
|Niemcy|   Peacock|     29.07.2003|       10260|1504.65|
|Niemcy|   Peacock|     30.07.2003|       10261|  448.0|
|Niemcy|  Callahan|     25.07.2

In [69]:
from pyspark.ml.feature import Bucketizer
splits = [0, 1000, 5000, 10000, 20000, float("inf")]
bucketizer = Bucketizer(splits=splits, inputCol="Utarg", outputCol="UtargBucket")

# Bucketize
bucketed_df = bucketizer.transform(df)

bucketed_agg = bucketed_df.groupBy("UtargBucket").sum("Utarg")
bucketed_agg.show()

+-----------+------------------+
|UtargBucket|        sum(Utarg)|
+-----------+------------------+
|        0.0|195967.73999999993|
|        1.0|          769990.4|
|        3.0|         120021.13|
|        2.0|142348.12999999998|
+-----------+------------------+



In [70]:
non_bucketed_agg = df.groupBy().sum("Utarg")
non_bucketed_agg.show()

+----------+
|sum(Utarg)|
+----------+
| 1228327.4|
+----------+



In [95]:
df.write.partitionBy("Kraj").option("header", True).mode("overwrite").csv("output/partitioned_by_country")

In [96]:
df.write.partitionBy("Sprzedawca").option("header", True).mode("overwrite").csv("output/partitioned_by_seller")

In [97]:
%%time
agg_original = df.filter(col("Kraj") == "Polska").groupBy("Sprzedawca").sum("Utarg")
agg_original.show()

+----------+------------------+
|Sprzedawca|        sum(Utarg)|
+----------+------------------+
|      King|116962.99000000002|
|     Dudek|          75048.04|
|   Sowiski| 72527.63000000002|
|  Kowalski|          68792.25|
+----------+------------------+

CPU times: user 3.98 ms, sys: 4.27 ms, total: 8.25 ms
Wall time: 188 ms


In [98]:
partitioned_df = spark.read.csv("output/partitioned_by_country", header=True, inferSchema=True)


In [99]:

partitioned_df.show(5)

+----------+---------------+------------+------+------+
|Sprzedawca|Data zamowienia|idZamowienia| Utarg|  Kraj|
+----------+---------------+------------+------+------+
|   Peacock|     12.07.2003|       10250|1552.6|Niemcy|
| Leverling|     15.07.2003|       10251|654.06|Niemcy|
|   Peacock|     11.07.2003|       10252|3597.9|Niemcy|
| Leverling|     16.07.2003|       10253|1444.8|Niemcy|
| Leverling|     17.07.2003|       10256| 517.8|Niemcy|
+----------+---------------+------------+------+------+
only showing top 5 rows



In [100]:
agg_partitioned = partitioned_df.filter(col("Kraj") == "Polska").groupBy("Sprzedawca").sum("Utarg")
agg_partitioned.show()

+----------+------------------+
|Sprzedawca|        sum(Utarg)|
+----------+------------------+
|      King|116962.99000000002|
|     Dudek|          75048.04|
|   Sowiski| 72527.63000000002|
|  Kowalski|          68792.25|
+----------+------------------+



In [104]:
df.show(2)

+------+----------+---------------+------------+------+
|  Kraj|Sprzedawca|Data zamowienia|idZamowienia| Utarg|
+------+----------+---------------+------------+------+
|Polska|  Kowalski|     16.07.2003|       10248| 440.0|
|Polska|   Sowiski|     10.07.2003|       10249|1863.4|
+------+----------+---------------+------------+------+
only showing top 2 rows

